## Table of Contents

1. [Import Libraries](#1)<br>
2. [Inception Model](#2)<br>
3. [ResNet Model](#3)<br>

# 🟢Import Libraries <a id ="1" ><a>

In [ ]:
import cv2
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

In [ ]:
import PIL
from tensorflow.keras.layers import Dense,Flatten,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam ,SGD
from sklearn.metrics import mean_squared_error as MSE_SKL
from sklearn.metrics import mean_absolute_error as MAE_SKL
from tensorflow.keras.metrics import mean_absolute_error as MAE_TF
from tensorflow.keras.regularizers import L2

In [ ]:
print(plt.style.available)
plt.style.use('ggplot')

## 🟢 Create Training Data  

In [ ]:
train_inputImg = pd.read_csv('../input/cephalometric-landmarks/train_senior.csv', delimiter=',')

train_images = []
train_images_rgb_blured = []

train_images_hsv = []
train_images_hsv_blured  = []

train_coords=[]



for row in train_inputImg.values:
    image_object = list(row)
    coords = image_object[1:]
    train_x_1_coord = []
    train_y_1_coord = []
    for i in range(0,len(coords)-1,2):
        train_x_1_coord.append((coords[i])/10)
        train_x_1_coord.append((coords[i+1])/10)
    train_coords.append(np.array(train_x_1_coord))
    
    gray_image = cv2.imread('../input/cephalometric-landmarks/cepha400/cepha400/' + image_object[0],cv2.IMREAD_GRAYSCALE ) 
    #print(gray_image.shape)
    gray_resized_image = cv2.resize(gray_image,(193,240))
    gray_resized_image = cv2.normalize(gray_resized_image, None, 0, 1.0,cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    #print(f"max = {np.max(gray_resized_image,axis=0)} , min = {np.min(gray_resized_image,axis=0)} ")
    rgb_resized_image = cv2.cvtColor(gray_resized_image,cv2.COLOR_GRAY2RGB)
    blured_rgb_resized_image = cv2.blur(rgb_resized_image,(3,3))
    
    hsv_resized_image = cv2.cvtColor(rgb_resized_image, cv2.COLOR_BGR2HSV)
    blured_hsv_resized_image = cv2.blur(hsv_resized_image ,(3,3))
    
    train_images.append(rgb_resized_image)
    train_images_rgb_blured.append(blured_rgb_resized_image)
    
    train_images_hsv.append(hsv_resized_image)
    train_images_hsv_blured.append(blured_hsv_resized_image)
    

In [ ]:
blur = cv2.blur(train_images[0],(3,3))
plt.imshow(blur)

In [ ]:
print("Type of train_images", type(train_images))
print("Shape of Training Images",(np.array(train_images).shape),"\n")

print("Type of train_images", type(train_images_hsv))
print("Shape of Training Images",(np.array(train_images_hsv).shape),"\n")

print("Type of train_coords", type(train_coords))
print("Shape of train_coords",(np.array(train_coords).shape))

## 🟢 Create Validation Data  

In [ ]:
val_inputImg = pd.read_csv('../input/cephalometric-landmarks/test1_senior.csv', delimiter=',')
val_images = []
train_images_rgb_blured = []
train_images_hsv_blured  = []


val_images_hsv = []
val_images_rgb_blured = []
val_images_hsv_blured  = []

val_coords = []

count = 1

for row in val_inputImg.values:
    image_object = list(row)
    coords = image_object[1:]
    train_x_1_coord = []
    train_y_1_coord = []
    val_x_1_coord = []
    val_y_1_coord = []
    
    gray_image = cv2.imread('../input/cephalometric-landmarks/cepha400/cepha400/' + image_object[0],cv2.IMREAD_GRAYSCALE ) 
    gray_resized_image = cv2.resize(gray_image,(193,240))
    gray_resized_image = cv2.normalize(gray_resized_image, None, 0, 1.0,cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    rgb_resized_image = cv2.cvtColor(gray_resized_image,cv2.COLOR_GRAY2RGB)
    blured_rgb_resized_image = cv2.blur(rgb_resized_image,(3,3))
    
    hsv_resized_image = cv2.cvtColor(rgb_resized_image, cv2.COLOR_BGR2HSV)
    blured_hsv_resized_image = cv2.blur(hsv_resized_image ,(3,3))
    
    if count <= 100:
            train_images.append(rgb_resized_image)
            train_images_hsv.append(hsv_resized_image)
            train_images_rgb_blured.append(blured_rgb_resized_image)
            train_images_hsv_blured.append(blured_hsv_resized_image)

    else:
            val_images.append(rgb_resized_image)
            val_images_hsv.append(hsv_resized_image)
            val_images_rgb_blured.append(blured_rgb_resized_image)
            val_images_hsv_blured.append(blured_hsv_resized_image)
            
    for i in range(0,len(coords)-1,2):
        
        if count <= 100:
            train_x_1_coord.append((coords[i])/10)
            train_x_1_coord.append((coords[i+1])/10)
        
        else:
            val_x_1_coord.append((coords[i])/10)
            val_x_1_coord.append((coords[i+1])/10)
    if count<=100:
        train_coords.append(np.array(train_x_1_coord))
    else:
        val_coords.append(np.array(val_x_1_coord))

    
    count += 1
    

In [ ]:
print("Type of val_images", type(val_images))
print("Shape of val_images ",(np.array(val_images).shape),"\n")

print("Type of val_images", type(val_images_hsv))
print("Shape of val_images ",(np.array(val_images_hsv).shape),"\n")

print("Type of val_coords", type(val_coords))
print("Shape of val_coords",(np.array(val_coords).shape))

## 🟢 Import Testing Data

In [ ]:
test_inputImg = pd.read_csv('../input/cephalometric-landmarks/test2_senior.csv', delimiter=',')
test_images = []
test_images_rgb_blured = []

test_images_hsv = []
test_images_hsv_blured = []

test_coords = []



for row in test_inputImg.values:
    image_object = list(row)
    coords = image_object[1:]
    test_x_1_coord = []
    for i in range(0,len(coords)-1,2):
        test_x_1_coord.append((coords[i])/10)
        test_x_1_coord.append((coords[i+1])/10)
    test_coords.append(np.array(test_x_1_coord))
    gray_image = cv2.imread('../input/cephalometric-landmarks/cepha400/cepha400/' + image_object[0],cv2.IMREAD_GRAYSCALE ) 
    gray_resized_image = cv2.resize(gray_image,(193,240))
    gray_resized_image = cv2.normalize(gray_resized_image, None, 0, 1.0,cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    rgb_resized_image = cv2.cvtColor(gray_resized_image,cv2.COLOR_GRAY2RGB)
    blured_rgb_resized_image = cv2.blur(rgb_resized_image,(3,3))
    
    hsv_resized_image = cv2.cvtColor(rgb_resized_image, cv2.COLOR_BGR2HSV)
    blured_hsv_resized_image = cv2.blur(hsv_resized_image ,(3,3))

    
    test_images.append(rgb_resized_image)
    test_images_rgb_blured.append(blured_rgb_resized_image)

    test_images_hsv.append(hsv_resized_image)
    test_images_hsv_blured.append(blured_hsv_resized_image)

In [ ]:
print("Type of test_images", type(test_images))
print("Shape of test_images ",(np.array(test_images).shape),"\n")

print("Type of test_images", type(test_images_hsv))
print("Shape of test_images ",(np.array(test_images_hsv).shape),"\n")

print("Type of test_coords", type(test_coords))
print("Shape of test_coords",(np.array(test_coords).shape))

## Final Sets Shape 

In [ ]:
print("********* Training Set *********")
print("Shape of Training Images",(np.array(train_images).shape),"\n")
print("Shape of Training Images",(np.array(train_images_hsv).shape),"\n")
print("Shape of train_coords",(np.array(train_coords).shape),"\n")

print("********* Validation Set *********")
print("Shape of val_images ",(np.array(val_images).shape),"\n")
print("Shape of test_images ",(np.array(val_images_hsv).shape),"\n")
print("Shape of val_coords",(np.array(val_coords).shape),"\n")

print("********* Test Set *********")
print("Shape of test_images ",(np.array(test_images).shape),"\n")
print("Shape of test_images ",(np.array(test_images_hsv).shape),"\n")
print("Shape of test_coords",(np.array(test_coords).shape))



## 🟢 Stack Training Data

In [ ]:
train_images_stacked = np.stack(train_images, axis=0)
train_images_rgb_blured_stacked = np.stack(train_images_rgb_blured, axis=0)


train_images_hsv_stacked = np.stack(train_images_hsv, axis=0)
train_images_hsv_blured_stacked = np.stack(train_images_hsv_blured, axis=0)



train_labels = np.array(train_coords)

In [ ]:
img = cv2.circle(train_images_stacked[0], (int(train_labels[0][0]),int(train_labels[0][1])), 2, (255,0,0),-1)
plt.imshow(img)


In [ ]:
img_ = cv2.circle(train_images_stacked[0], (800,1080), 25, (255,0,0),-1)
plt.imshow(img_)

## 🟢 Stack Validation Data

In [ ]:
val_images_stacked= np.stack(val_images, axis=0)
val_images_rgb_blured_stacked= np.stack(val_images_rgb_blured, axis=0)


val_images_hsv_stacked= np.stack(val_images_hsv, axis=0)
val_images_hsv_blured_stacked= np.stack(val_images_hsv_blured, axis=0)

val_labels = np.array(val_coords)

val_data = (val_images_stacked,val_labels)
val_data_rgb_blured = (val_images_rgb_blured_stacked,val_labels)                                        

val_data_hsv = (val_images_hsv_stacked,val_labels)
val_data_hsv_blured = (val_images_hsv_blured_stacked,val_labels)



## 🟢 Stack Test Data

In [ ]:
test_images_stacked= np.stack(test_images, axis=0)
test_images_rgb_blured_stacked= np.stack(test_images_rgb_blured, axis=0)

test_images_hsv_stacked= np.stack(test_images_hsv, axis=0)
test_images_hsv_blured_stacked= np.stack(test_images_hsv_blured, axis=0)


test_labels = np.array(test_coords)

## Final Stacked Shape

In [ ]:
print("********* Stacked Training Set *********")
print("Type of train_images_stacked",(type(train_images_stacked)))
print("Shape of train_images_stacked",((train_images_stacked).shape))
print("Shape of train_images_stacked",((train_images_rgb_blured_stacked).shape))
print("Shape of train_images_stacked",((train_images_hsv_blured_stacked).shape))
print("Type of train_labels",(type(train_labels)))
print("Shape of train_labels",((train_labels).shape),"\n")

print("********* Stacked Validation Set *********")
print("Shape of val_images_stacked ",((val_images_stacked).shape))
print("Shape of val_labels",((val_labels).shape))
print("Val_datet => (val_labels,val_labels)")
print("Type of val_data",(type(val_data)),"\n")

print("********* Stacked Test Set *********")
print("Type of test_labels",(type(test_labels)))
print("Shape of test_images_stacked ",((test_images_stacked).shape))
print("Type of test_labels",(type(test_labels)))
print("Shape of test_labels",((test_labels).shape))

# Radial Error Loss Metric

In [ ]:
# def mean_radial_error(X_Y_True,X_Y_Predicred):
#     delta_x_2 =  np.power( ( np.subtract(X_Y_True[:,0,:] , X_Y_Predicred[:,0,:]) ) , 2) #(2,4)
#     delta_y_2 =  np.power( ( np.subtract(X_Y_True[:,1,:] , X_Y_Predicred[:,1,:]) ) , 2) #(2,4)

#     radial_error_lm_ex = np.sqrt(np.add(delta_x_2,delta_y_2)) #(2,4)

#     radial_error_ex = np.sum(radial_error_lm_ex, axis = 1) #(2,)

#     radial_error =  np.sum(radial_error_ex, axis = 0)

#     mean_radial_error = radial_error / len(radial_error_ex)

    
#     return mean_radial_error
    

## Mean Radial Error For each Landmark

In [ ]:
# def mean_radial_error_landmark(x_y_true,x_y_pred):
#     delta_x_2 =  np.power( ( np.subtract(x_y_true[:,0,:] , x_y_pred[:,0,:]) ) , 2) #(2,4)
#     delta_y_2 =  np.power( ( np.subtract(x_y_true[:,1,:] , x_y_pred[:,1,:]) ) , 2) #(2,4)

#     radial_error_lm_ex = np.sqrt(np.add(delta_x_2,delta_y_2)) #(2,4)

# #     print(radial_error_lm_ex)

#     radial_error_lm = np.sum(radial_error_lm_ex, axis = 0,keepdims=True) #(1,4)
# #     print(radial_error_lm)

#     mean_radial_error_lm = radial_error_lm / x_y_true.shape[0]
#     print(x_y_true.shape[0])
    
#     return mean_radial_error_lm

In [ ]:
def mean_radial_error_landmark(x_true,y_true,x_pred,y_pred):
    delta_x_2 =  np.power( ( np.subtract(x_true , x_pred) ) , 2) #(2,4)
    delta_y_2 =  np.power( ( np.subtract(y_true, y_pred ) ) , 2) #(2,4)

    radial_error_lm_ex = np.sqrt(np.add(delta_x_2,delta_y_2)) #(2,4)

#     print(radial_error_lm_ex)

    radial_error_lm = np.sum(radial_error_lm_ex, axis = 0,keepdims=True) #(1,4)
#     print(radial_error_lm)

    mean_radial_error_lm = radial_error_lm / x_true.shape[0]
    print(x_true.shape[0])
    
    return radial_error_lm_ex,mean_radial_error_lm

## 🔵 Create InceptionResNetV2 Model  <a id="2"></a>

In [ ]:
from keras.models import Model

pretrained_model= tf.keras.applications.InceptionResNetV2(include_top=False,
                   input_shape=(240,193,3),
                                                          
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False
# add a global spatial average pooling layer
x = pretrained_model.output
x =  tf.keras.layers.Flatten()(x)
# let's add a fully-connected layer
#x = Dense(1024, activation='relu')(x)
#x =Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x =Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x =Dropout(0.3)(x)
# and a linear output layer
prediction = Dense(38, activation='linear')(x)
# this is the model we will train
InceptionResNetV2 = Model(inputs=pretrained_model.input, outputs=prediction)


## 🔵 Compile Model

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.1,
                                                patience=5,
                                                min_delta=0.001)

InceptionResNetV2.compile(optimizer=Adam(learning_rate=0.4),loss='mae',metrics=['mae'])

## 🔵 Fit Model

In [ ]:
start_time = time.time()
history = InceptionResNetV2.fit(train_images_hsv_stacked,train_labels, validation_data = val_data_hsv, epochs=400)
end_time = time.time()
print("Training_time:",end_time-start_time)

In [ ]:
plt.plot(history.history['loss'], label = "Training Loss")
plt.plot(history.history['val_loss'], label = "Val Loss")
plt.legend()
plt.show()

## 🔵 Make Predictions


In [ ]:
start_time = time.time()

prediction_IncepRes=InceptionResNetV2.predict(test_images_stacked)

end_time =  time.time()

print("prediction_time:",end_time-start_time)


In [ ]:
MAE_SKL(test_labels, prediction_IncepRes)

In [ ]:
x_coored_ind = np.arange(start=0, stop=38, step=2)
y_coored_ind = np.arange(start=1, stop=39, step=2)

x_true = test_labels[:,x_coored_ind]
y_true = test_labels[:,y_coored_ind]

x_pred_incep = prediction_IncepRes[:,x_coored_ind]
y_pred_incep = prediction_IncepRes[:,y_coored_ind]

mre_each_lm_incep=(mean_radial_error_landmark(x_true,y_true,x_pred_incep,y_pred_incep)).reshape(19)

In [ ]:
delta_x_incep = ( np.subtract(x_true , x_pred_incep) )
delta_x_lm_incep = np.sum(delta_x_incep, axis = 0,keepdims=True)/100
difference_x_incep=(delta_x_lm_incep).reshape(19)

delta_y_incep = ( np.subtract(y_true , y_pred_incep) )
delta_y_lm_incep = np.sum(delta_y_incep, axis = 0,keepdims=True)/100
difference_y_incep =(delta_y_lm_incep).reshape(19)

In [ ]:
landmark = np.array(np.arange(1,20),dtype=int)
import pandas as pd

results_incep = pd.DataFrame({"landmark":landmark,
                        "Mean Radial Error":mre_each_lm_incep,
                        "X_true - X_pred":difference_x_incep,
                        "y_true - y_pred":difference_y_incep})
results_incep.set_index(['landmark'])

In [ ]:
for i in range(len(test_labels)):
    print(f'mae of image {i}  = {mean_absolute_error(test_labels[i], prediction_IncepRes[i])}')

In [ ]:
img_number = 56
img = np.copy(test_images_stacked[img_number])
for i in range(0,len(test_labels[img_number])-1,2):
    img = cv2.circle(img, (int(test_labels[img_number][i]),int(test_labels[img_number][i+1])), 2, (255,0,0),-1)
    img = cv2.circle(img, (int(prediction_IncepRes[img_number][i]),int(prediction_IncepRes[img_number][i+1])), 2, (0,255,0),-1)
plt.imshow(img)

In [ ]:
img_number = 55
img = np.copy(test_images_stacked[img_number])
for i in range(0,len(test_labels[img_number])-1,2):
    img = cv2.circle(img, (int(test_labels[img_number][i]),int(test_labels[img_number][i+1])), 2, (255,0,0),-1)
    img = cv2.circle(img, (int(prediction_IncepRes[img_number][i]),int(prediction_IncepRes[img_number][i+1])), 2, (0,255,0),-1)
plt.imshow(img)

In [ ]:
img_number = 68
img = np.copy(test_images_stacked[img_number])
for i in range(0,len(test_labels[4])-1,2):
    img = cv2.circle(img, (int(test_labels[img_number][i]),int(test_labels[img_number][i+1])), 2, (255,0,0),-1)
    img = cv2.circle(img, (int(prediction_IncepRes[img_number][i]),int(prediction_IncepRes[img_number][i+1])), 2, (0,255,0),-1)
plt.imshow(img)

In [ ]:
InceptionResNetV2.save('incep_adabtiveLR.h5')

In [ ]:
from IPython.display import FileLink
FileLink("./incep_adabtiveLR.h5")

## 🟡 K-fold Cross validation<a id="3"></a>

In [ ]:
inputs = np.concatenate((train_images_stacked, val_images_stacked), axis=0)
targets = np.concatenate((train_labels, val_labels), axis=0)
print(inputs.shape)
print(targets.shape)

In [ ]:
from sklearn.model_selection import KFold
from keras.models import Model
import tensorflow_addons as tfa
# Merge inputs and targets
inputs = np.concatenate((train_images_stacked, val_images_stacked), axis=0)
targets = np.concatenate((train_labels, val_labels), axis=0)
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)
# K-fold Cross Validation model evaluation
fold_no = 1
# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []
for train, val in kfold.split(inputs, targets):
  # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

  # Fit data to model
#   history = model.fit(inputs[train], targets[train],
#               batch_size=batch_size,
#               epochs=no_epochs,
#               verbose=verbosity)


    pretrained_model= tf.keras.applications.InceptionResNetV2(include_top=False,
                       input_shape=(240,193,3),
                        pooling='avg',
                       weights='imagenet')
    for layer in pretrained_model.layers:
            layer.trainable=False
    # add a global spatial average pooling layer
    x = pretrained_model.output
    x =  tf.keras.layers.Flatten()(x)
    x =Dropout(0.25)(x)
    x = Dense(1024, activation='relu',kernel_initializer = 'he_normal')(x)
#     x =Dropout(0.25)(x)
    x = Dense(512, activation='relu',kernel_initializer = 'he_normal')(x)
#     x =Dropout(0.25)(x)
    x = Dense(256, activation='relu',kernel_initializer = 'he_normal')(x)
#     x =Dropout(0.25)(x)
    x = Dense(128, activation='relu',kernel_initializer = 'he_normal')(x)
    x =Dropout(0.25)(x)
    # and a linear output layer
    prediction = Dense(38, activation='linear')(x)
    # this is the model we will train
    InceptionResNetV2 = Model(inputs=pretrained_model.input, outputs=prediction)
    clr = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=10e-4,
    maximal_learning_rate=0.1,
    scale_fn=lambda x: 1/(2.**(x-1)),
    step_size=2 * 32)
    
    InceptionResNetV2.compile(optimizer=Adam(clr),loss='mae',metrics=['mae'])
    history = InceptionResNetV2.fit(inputs[train], targets[train], validation_data = (inputs[val], targets[val]), batch_size = 32,epochs=400) 

  # Generate generalization metrics
    scores = InceptionResNetV2.evaluate(test_images_stacked, test_labels, verbose=0)
    print(f'Score for fold {fold_no}: {InceptionResNetV2.metrics_names[0]} of {scores[0]}; {InceptionResNetV2.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    InceptionResNetV2.save(f"InceptionV2_fold{fold_no}")

  # Increase fold number
    fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

In [ ]:
from sklearn.model_selection import KFold
from keras.models import Model
import tensorflow_addons as tfa
# Merge inputs and targets
inputs = np.concatenate((train_images_stacked, val_images_stacked), axis=0)
targets = np.concatenate((train_labels, val_labels), axis=0)
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)
# K-fold Cross Validation model evaluation
fold_no = 1
# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []
for train, val in kfold.split(inputs, targets):
  # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

  # Fit data to model
#   history = model.fit(inputs[train], targets[train],
#               batch_size=batch_size,
#               epochs=no_epochs,
#               verbose=verbosity)



    pretrained_model2= tf.keras.applications.ResNet50(include_top=False,
                       input_shape=(240,193,3),
                        pooling='avg',
                       weights='imagenet')

    for layer in pretrained_model2.layers:
            layer.trainable=False
    # add a global spatial average pooling layer
    x = pretrained_model2.output
    x =  tf.keras.layers.Flatten()(x)
    # let's add a fully-connected layer
    x =Dropout(0.25)(x)
    x = Dense(1024, activation='relu',kernel_initializer = 'he_normal')(x)
#     x =Dropout(0.25)(x)
    x = Dense(512, activation='relu',kernel_initializer = 'he_normal')(x)
#     x =Dropout(0.25)(x)
    x = Dense(256, activation='relu',kernel_initializer = 'he_normal')(x)
#     x =Dropout(0.25)(x)
    x = Dense(128, activation='relu',kernel_initializer = 'he_normal')(x)
    x =Dropout(0.25)(x)
    # and a linear output layer
    prediction = Dense(38, activation='linear')(x)
    # this is the model we will train
    resnet50_model = Model(inputs=pretrained_model2.input, outputs=prediction)
    clr = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=10e-4,
    maximal_learning_rate=0.1,
    scale_fn=lambda x: 1/(2.**(x-1)),
    step_size=2 * 32)
    
    resnet50_model.compile(optimizer=Adam(clr),loss='mae',metrics=['mae'])
    history = resnet50_model.fit(inputs[train], targets[train], validation_data = (inputs[val], targets[val]), batch_size = 32,epochs=400) 

  # Generate generalization metrics
    scores = resnet50_model.evaluate(test_images_stacked, test_labels, verbose=0)
    print(f'Score for fold {fold_no}: {resnet50_model.metrics_names[0]} of {scores[0]}; {resnet50_model.metrics_names[1]} of {scores[1]}%')
    acc_per_fold.append(scores[1] )
    loss_per_fold.append(scores[0])
    resnet50_model.save(f"resnet50_model_fold{fold_no}.h5")

  # Increase fold number
    fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

## 🟡 Create ResNet50 model <a id="3"></a>

In [ ]:
from keras.models import Model
import tensorflow_addons as tfa
pretrained_model2= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(240,193,3),                    
                   weights='imagenet')

for layer_no , layer in zip(range(1,len(pretrained_model2.layers)+1),pretrained_model2.layers): #176 Layers
#     if layer_no > 172:
#         layer.trainable=True
#     else:
        layer.trainable=False
    
#     print(layer_no)
x = pretrained_model2.output
x =  tf.keras.layers.Flatten()(x)

initializer = tf.keras.initializers.RandomUniform(minval=0., maxval=1.) #kernel_initializer=initializer


# x = Dense(2048, activation='relu', kernel_initializer = 'he_normal')(x)
x = Dense(1024, activation='relu', kernel_initializer = 'he_normal')(x)
# # x =Dropout(0.25)(x)
# x = Dense(512, activation='relu', kernel_initializer = 'he_normal')(x)
# # x =Dropout(0.25)(x)
# x = Dense(256, activation='relu', kernel_initializer = 'he_normal')(x)
# # x =Dropout(0.25)(x)
# x = Dense(128, activation='relu', kernel_initializer = 'he_normal')(x)
# # x =Dropout(0.25)(x)

#x = Dense(512, activation='relu',kernel_initializer=initializer)(x)

# x = Dense(128, activation='relu')(x)

prediction = Dense(38, activation='linear')(x)

resnet50_model = Model(inputs=pretrained_model2.input, outputs=prediction)


In [ ]:
resnet50_model.summary()

## 🟡 Adabtive Learning Rate

In [ ]:
!pip install -q -U tensorflow_addons

In [ ]:
import tensorflow_addons as tfa
clr = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=10e-2,
    maximal_learning_rate=0.5,
    scale_fn=lambda x: 1/(2.**(x-1)),
    step_size=2 * 64
)

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.1,
                                                patience=5,
                                                min_delta=0.002)

In [ ]:
checkpoint_filepath = '/resnet/checkpoint'

resnet_model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

## 🟡 Compile Model

In [ ]:
resnet50_model.compile(optimizer=Adam(learning_rate = 0.1),loss='mae',metrics=['mae'])


## 🟡 Fit Model

In [ ]:
start_time = time.time()

history = resnet50_model.fit(train_images_stacked,train_labels, validation_data = val_data, epochs=400,batch_size =64,)

end_time =  time.time()

print("Trining_time: ",end_time-start_time )

In [ ]:
plt.plot(history.history['loss'], label = "Training Loss")
plt.plot(history.history['val_loss'], label = "Val Loss")
plt.legend()
plt.show()

In [ ]:
step = np.arange(0, 100 * 64)
# lr = clr(step)
plt.plot(step, clr)
plt.xlabel("Steps")
plt.ylabel("Learning Rate")
plt.show()

## 🟡 Make Predictions

In [ ]:
start_time = time.time()

prediction_ResNet50=resnet50_model.predict(test_images_stacked)

end_time =  time.time()

print("prediction_time: ",end_time-start_time )

In [ ]:
print("MSE: ",MSE_SKL(test_labels, prediction_ResNet50))
print("MAE: ",MAE_SKL(test_labels, prediction_ResNet50))

## Mean Radial Error for each LandMark along Test Set

In [ ]:
x_coored_ind = np.arange(start=0, stop=38, step=2)
y_coored_ind = np.arange(start=1, stop=39, step=2)

x_true = test_labels[:,x_coored_ind]
y_true = test_labels[:,y_coored_ind]

x_pred = prediction_ResNet50[:,x_coored_ind]
y_pred = prediction_ResNet50[:,y_coored_ind]

radial_error_lm_ex_resnet , mre_each_lm_resnet = (mean_radial_error_landmark(x_true,y_true,x_pred,y_pred))
mre_each_lm_resnet = mre_each_lm_resnet.reshape(19)

## Radial Error for each LandMark across 100 test image

In [ ]:
import seaborn as sns
plt.figure(figsize=(15,8))
sns.boxplot(data = radial_error_lm_ex_resnet[:,:])
plt.xlabel("LandMarks")
plt.ylabel("Error mm")
plt.title("Radial Error for each LandMark across 100 test image")

In [ ]:
delta_x = ( np.subtract(x_true , x_pred) )
delta_x_lm = np.sum(delta_x, axis = 0,keepdims=True)/100
difference_x=(delta_x_lm).reshape(19)

delta_y = ( np.subtract(y_true , y_pred) )
delta_y_lm = np.sum(delta_y, axis = 0,keepdims=True)/100
difference_y =(delta_y_lm).reshape(19)


In [ ]:
import pandas as pd
landmark = np.array(np.arange(1,20),dtype=int)
results = pd.DataFrame({"landmark":landmark,
                        "Mean Radial Error":mre_each_lm_resnet,
                        "X_true - X_pred":difference_x,
                        "y_true - y_pred":difference_y})
results.set_index(['landmark'])


## Mean Radial Error For Each LandMark

In [ ]:
mre_df = results[["Mean Radial Error"]]
mre_df.plot(kind = "bar",figsize=(8,5))
# mre_df

In [ ]:
# mre_each_lm_incep - mre_each_lm_resnet
# print("Total MRE of Incep - Total MRE of ResNet: ",np.round(np.sum(mre_each_lm_incep)-np.sum(mre_each_lm_resnet),3))

In [ ]:
# for i in range(len(test_labels)):
#     print(f'mae of image {i}  = {mean_absolute_error(test_labels[i], prediction_ResNet50[i])}')
    

In [ ]:
mae_ =mean_absolute_error(test_labels, prediction_ResNet50)

print("MAE: ",mae_)

mae_list= list(mae_)

max_mae = max(mae_list)
min_mae = min(mae_list)
mid_mae = 9.303155 ##Image_Index = 58
# median_mae = np.median(mae_list)


max_mae_index    = mae_list.index(max_mae)
min_mae_index    = mae_list.index(min_mae)
# mid_mae_index    = mae_list.index(mid_mae)   
# median_mae_index = mae_list.index(median_mae)

print("Max Error: {} , Index of Max Error: {}".format(max_mae , max_mae_index))
print("Min Error: {} , Index of Min Error: {}".format(min_mae , min_mae_index))
# print("Median Error: {} , Index of Median Error: {}".format(mid_mae , mid_mae_index))



In [ ]:

fig = plt.figure(figsize=(20, 20))
rows = 1
columns = 3

for j ,img_number,title in zip(range(1,4),[56,68,55],["Max MAE","Mid MAE","Min MAE"]):
    img =np.copy(test_images[img_number])
    for i in range(0,38,2):
            img_ = cv2.circle(img,(int(prediction_ResNet50[img_number][i]),int(prediction_ResNet50[img_number][i+1])), 2, (255,0,0),-1)
            img_ = cv2.circle(img,(int(test_labels[img_number][i]),int(test_labels[img_number][i+1])), 2, (0,255,0),-1)
    fig.add_subplot(rows, columns, j)
    plt.imshow(img_)
    plt.title(title)


In [ ]:
resnet50_model.save('resnet50_adabtiveLR.h5')

In [ ]:
resnet50_model.save('resnet50_adabtiveLR')

In [ ]:
from IPython.display import FileLink
FileLink("./resnet50_adabtiveLR.h5")

In [ ]:
from IPython.display import FileLinks
FileLinks(r"./resnet50_adabtiveLR")